In [1]:
from flask import Flask
from flask import request
import numpy as np
import pandas as pd
from scipy.stats.stats import pearsonr # used to calculate correlation coefficient
from pymongo import MongoClient
import json
from datetime import datetime, timedelta
from pprint import pprint
from enum import Enum
import copy
from collections import deque
import time
import csv

In [2]:
def reconstruct_currency_date(cur):
    frmt = "{:%Y-%m-%d}"
    return_frequency = 'hourly'
    if return_frequency == 'hourly':
        frmt = "{:%Y-%m-%d %H:%M:%S}"
    for cur_value, index in zip(cur['history'], range(len(cur['history']))):
        #  cur['history'][index]['date'] = datetime.fromtimestamp(cur_value['time'])
        cur['history'][index]['date'] = frmt.format(datetime.fromtimestamp(cur_value['time']))
    return cur

In [9]:
client = MongoClient('localhost',
                    authSource='bitcoin')
db = client.bitcoin
daily_data = db.daily_data

data1 = reconstruct_currency_date(daily_data.find_one({'Ccy': 'BTC'}))

df_data1 = pd.DataFrame(data1['history'][:1000])
# pprint(df_data1)

df_data1 = df_data1.set_index(['date'])         # index: string
df_data1.index = pd.to_datetime(df_data1.index)


btc = df_data1.pct_change()
a = btc['close'].values[1:2000]

data2 = reconstruct_currency_date(daily_data.find_one({'Ccy': 'ETH'}))

df_data2 = pd.DataFrame(data2['history'][:1000])

df_data2 = df_data2.set_index(['date'])         # index: string
df_data2.index = pd.to_datetime(df_data2.index)

eth = df_data2.pct_change()
b = eth['close'].values[1:2000]

correlation = pearsonr(a, b)

pprint(correlation)
# pprint(b)

(-0.0530165495705125, 0.09397897132024564)
